In [1]:
import sys
sys.path.insert(0,'../')

import network
reload(network)

net = network.Net()

Using gpu device 1: Tesla K40c (CNMeM is disabled)


In [2]:
import data_loader
reload(data_loader)

from data_loader import DataLoader

classes = [wnid.strip() for wnid in open('../synsets.txt').readlines()]
dataset = DataLoader(classes, '../../ilsvrc2012/train_resized','../../ilsvrc2012/val_resized')

In [5]:
from PIL import Image
import preprocessor
reload(preprocessor)
from preprocessor import Fetcher
import tree_distance
reload(tree_distance)
from tree_distance import Distance

fetcher = Fetcher('center',net.MEAN_IMAGE)
distance = Distance('../wordnet.is_a.txt','../synsets.txt')

In [ ]:
from multiprocessing import Pool, Process, Queue, Value
import sys, datetime

def fetch(q,max_buf,flag,fetcher,distance) :
    while(flag.value != 0) :
        if( q.qsize() < max_buf ) :
            fetched1 = dataset.fetch_train(64)
            fetched2 = dataset.fetch_train(64)

            ims1 = fetcher.fetch(fetched1)
            ims2 = fetcher.fetch(fetched2)
            dist = distance.dist(fetched1,fetched2)
            q.put((ims1,ims2,dist))
            
#             import datetime
#             print 'Enqueued', datetime.datetime.now()
#             sys.stdout.flush()

            
q = Queue()
flag = Value('d', 1) 
p = Process(target=fetch,args=(q,10,flag,fetcher,distance))

p.start()
for i in range(100000):
    ims1,ims2,dist = q.get()
    loss, loss_before_clipped = net.train(ims1,ims2,dist)
    
#     import datetime
#     print 'Trained', datetime.datetime.now()
#     sys.stdout.flush()
    if i % 100 == 0 :
        print datetime.datetime.now(), i, loss, loss_before_clipped
        sys.stdout.flush()

flag.value = 0
p.join()